## 1. Connect to GBQ

In [ ]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import random

In [ ]:
# These first two values will be different on your machine. 
service_path = "C:\\users\\jmeese\\desktop\\ADA\\"
service_file = 'UMT-MSBA-a0366399b58f.json' # change this to your authentication information  
gbq_proj_id = 'umt-msba' # change this to your poroject. 

# And this should stay the same. 
private_key =service_path + service_file

In [ ]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [ ]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

## 2. List of Owners

In [ ]:
query = (
    "SELECT "
    "DISTINCT(card_no) "
    "FROM "
    "`umt-msba.wedge_example.daily_trans_*` "
    "WHERE card_no != 3.0"
)

In [ ]:
# And we execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

In [ ]:
all_owners = []

for row in query_job :
    all_owners.append(row[0])


In [ ]:
len(all_owners)

## 3. Sample

In [ ]:
owner_sample = random.sample(all_owners,400)

## 4. Extract records

In [ ]:
query = (
    "SELECT "
    "* "
    "FROM "
    "`umt-msba.wedge_example.daily_trans_*` "
    "WHERE "
    "card_no IN("
)

query = query + ",".join([str(num) for num in owner_sample]) + ")"

In [ ]:
# And we execute queries with `client.query`
query_job = client.query(
    query,
    location="US",
)

In [ ]:
query_job.result()

## 5. Write to Text

In [ ]:
headers = """datetime	register_no	emp_no	trans_no	upc	description	
             trans_type	trans_subtype	trans_status	department	
             quantity	Scale	cost	unitPrice	total	
             regPrice	altPrice	tax	taxexempt	foodstamp	
             wicable	discount	memDiscount	discountable	
             discounttype	voided	percentDiscount	ItemQtty	
             volDiscType	volume	VolSpecial	mixMatch	
             matched	memType	staff	numflag	itemstatus	
             tenderstatus	charflag	varflag	batchHeaderID	
             local	organic	display	receipt	card_no	store	
             branch	match_id	trans_id""".split()

In [ ]:
with open("owner_sample.txt",'w') as ofile :
    ofile.write("\t".join(headers) + "\n")
    
    for idx, row in enumerate(query_job) :
        ofile.write("\t".join([str(element) for element in row]) + "\n")
    
#        if idx == 100 :
#            break
